In [23]:
import cv2
import numpy as np
from keras.models import load_model

In [24]:
backSub = cv2.createBackgroundSubtractorMOG2()

lower_blue = np.array([100, 50, 50])
upper_blue = np.array([130, 255, 255])
lower_green = np.array([40, 50, 50])
upper_green = np.array([70, 255, 255])

In [25]:
cap = cv2.VideoCapture('field.mp4')

In [26]:
anomaly_counter = 0
offset = 10
while True:
    ret, frame = cap.read()
    if not ret:
        break
    fgMask = backSub.apply(frame)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask = cv2.bitwise_or(mask_blue, mask_green)
    mask = cv2.bitwise_not(mask)
    fgMask = cv2.bitwise_and(fgMask, mask)
    if np.any(fgMask):
        anomaly_counter += 1
        contours, _ = cv2.findContours(
            fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        centers = []
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            if w * h > 25:
                centers.append([x + w / 2, y + h / 2])
        if len(centers) > 0:
            clustering = DBSCAN(eps=50, min_samples=2).fit(centers)
            labels = clustering.labels_
            unique_labels = set(labels)
            for label in unique_labels:
                if label != -1:
                    cluster_centers = [centers[idx] for idx in range(
                        len(centers)) if labels[idx] == label]
                    min_x = int(
                        min(cluster_centers, key=lambda x: x[0])[0]) - offset
                    min_y = int(
                        min(cluster_centers, key=lambda x: x[1])[1]) - offset
                    max_x = int(
                        max(cluster_centers, key=lambda x: x[0])[0]) + offset
                    max_y = int(
                        max(cluster_centers, key=lambda x: x[1])[1]) + offset
                    cv2.rectangle(frame, (min_x, min_y),
                                  (max_x, max_y), (0, 255, 0), 2)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [27]:
if anomaly_counter > 0:
    print("Anomaly detected")
else:
    print("No anomaly detected")

Anomaly detected
